## Setup

In [1]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')
print("✓ Google Drive mounted")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Google Drive mounted


In [2]:
import os
os.chdir('/content')

# Clone repository
!rm -rf automatic-pain-recognition
!git clone https://github.com/alicka33/automatic-pain-recognition.git

os.chdir('/content/automatic-pain-recognition')
print(f"✓ Repository cloned. Current directory: {os.getcwd()}")

Cloning into 'automatic-pain-recognition'...
remote: Enumerating objects: 333, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 333 (delta 28), reused 44 (delta 16), pack-reused 277 (from 1)
Receiving objects: 100% (333/333), 12.41 MiB | 24.16 MiB/s, done.
Resolving deltas: 100% (152/152), done.
✓ Repository cloned. Current directory: /content/automatic-pain-recognition


In [3]:
# Install requirements
!pip install -q -r requirements.txt
!pip install -q pytest
print("✓ Dependencies installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.2 MB/s eta 0:00:00
✓ Dependencies installed


In [3]:
import sys
from pathlib import Path

# Add project to path
project_root = Path('/content/automatic-pain-recognition')
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✓ Project path updated")

✓ Project path updated


In [ ]:
# 1. Czyszczenie starego bałaganu
!pip uninstall -y mediapipe
!rm -rf /usr/local/lib/python3.12/dist-packages/mediapipe

# 2. Instalacja wersji zgodnej z 3.12 (bez cache)
!pip install --no-cache-dir mediapipe==0.10.14

# 3. WYMUSZENIE RESTARTU - to zabije sesję i odświeży ścieżki
import os
os.kill(os.getpid(), 9)

In [4]:
import numpy as np
import tempfile
import shutil
from pathlib import Path
from unittest.mock import Mock

from data_preparation.processing_pipeline_mediapipe import (
    load_reference_keypoints,
    create_face_mesh,
    parse_landmarks_from_results,
    procrustes_analysis,
    center_keypoints,
    keypoints_to_feature_vector
)

## Run Tests

In [5]:
print("\n" + "="*70)
print("TEST 1: load_reference_keypoints")
print("="*70)

try:
    # Create temporary reference keypoints file
    temp_dir = tempfile.mkdtemp()
    ref_file = os.path.join(temp_dir, 'reference_keypoints.npy')
    ref_keypoints = np.random.rand(478, 3).astype(np.float32)
    np.save(ref_file, ref_keypoints)

    # Load reference keypoints - returns tuple (array, bool)
    loaded, use_front = load_reference_keypoints(Path(ref_file), num_landmarks=478)

    assert isinstance(loaded, np.ndarray)
    assert loaded.shape == (478, 3)
    assert loaded.dtype in [np.float32, np.float64]
    assert use_front is True

    print(f"✓ load_reference_keypoints works")
    print(f"  Loaded shape: {loaded.shape}")
    print(f"  Use frontalization: {use_front}")

    # Clean up
    shutil.rmtree(temp_dir)
    print("\n✓ TEST 1 PASSED")
except Exception as e:
    print(f"✗ TEST 1 FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 1: load_reference_keypoints
✓ load_reference_keypoints works
  Loaded shape: (478, 3)
  Use frontalization: True

✓ TEST 1 PASSED


In [6]:
print("\n" + "="*70)
print("TEST 1c: load_reference_keypoints - Shape Mismatch")
print("="*70)

try:
    # Create file with wrong shape
    temp_dir = tempfile.mkdtemp()
    wrong_file = os.path.join(temp_dir, 'wrong_shape.npy')
    wrong_keypoints = np.random.rand(100, 3).astype(np.float32)  # Wrong size
    np.save(wrong_file, wrong_keypoints)

    # Try to load with expected 478 landmarks
    ref, use_front = load_reference_keypoints(Path(wrong_file), num_landmarks=478)

    assert ref is None
    assert use_front is False
    print(f"✓ Shape mismatch handled correctly")
    print(f"  Expected (478, 3), got (100, 3)")
    print(f"  Returns None and use_frontalization=False")

    # Clean up
    shutil.rmtree(temp_dir)
    print("\n✓ TEST 1c PASSED")
except Exception as e:
    print(f"✗ TEST 1c FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 1c: load_reference_keypoints - Shape Mismatch
✓ Shape mismatch handled correctly
  Expected (478, 3), got (100, 3)
  Returns None and use_frontalization=False

✓ TEST 1c PASSED


In [7]:
print("\n" + "="*70)
print("TEST 1b: load_reference_keypoints - Missing File")
print("="*70)

try:
    # Test handling of missing file
    ref, use_front = load_reference_keypoints(Path('/nonexistent/path.npy'), num_landmarks=478)

    assert ref is None
    assert use_front is False
    print(f"✓ Missing file handled correctly")
    print(f"  Returns None and use_frontalization=False")

    print("\n✓ TEST 1b PASSED")
except Exception as e:
    print(f"✗ TEST 1b FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 1b: load_reference_keypoints - Missing File
✓ Missing file handled correctly
  Returns None and use_frontalization=False

✓ TEST 1b PASSED


In [8]:
print("\n" + "="*70)
print("TEST 2: create_face_mesh")
print("="*70)

try:
    # Create face mesh
    face_mesh = create_face_mesh()

    assert face_mesh is not None
    print(f"✓ create_face_mesh works")
    print(f"  Face mesh created successfully")

    print("\n✓ TEST 2 PASSED")
except Exception as e:
    print(f"✗ TEST 2 FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 2: create_face_mesh
✓ create_face_mesh works
  Face mesh created successfully

✓ TEST 2 PASSED


In [9]:
print("\n" + "="*70)
print("TEST 3: parse_landmarks_from_results")
print("="*70)

try:
    # Create mock landmarks (478 landmarks with x, y, z coordinates)
    landmarks_list = []
    for i in range(478):
        landmark = Mock()
        landmark.x = np.random.rand()
        landmark.y = np.random.rand()
        landmark.z = np.random.rand()
        landmarks_list.append(landmark)

    # Create mock face landmarks object
    mock_face_landmarks = Mock()
    mock_face_landmarks.landmark = landmarks_list

    # Create mock results - multi_face_landmarks must be a LIST (subscriptable)
    mock_results = Mock()
    mock_results.multi_face_landmarks = [mock_face_landmarks]  # List, not Mock

    # Parse landmarks
    keypoints = parse_landmarks_from_results(mock_results, expected_num=478)

    assert isinstance(keypoints, np.ndarray)
    assert keypoints.shape == (478, 3)
    print(f"✓ parse_landmarks_from_results works")
    print(f"  Output shape: {keypoints.shape}")
    print(f"  X range: [{keypoints[:, 0].min():.4f}, {keypoints[:, 0].max():.4f}]")

    # Test with no detection
    mock_results_empty = Mock()
    mock_results_empty.multi_face_landmarks = None
    keypoints_empty = parse_landmarks_from_results(mock_results_empty, expected_num=478)
    assert keypoints_empty is None
    print(f"✓ No detection handled correctly (returns None)")

    print("\n✓ TEST 3 PASSED")
except Exception as e:
    print(f"✗ TEST 3 FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 3: parse_landmarks_from_results
✓ parse_landmarks_from_results works
  Output shape: (478, 3)
  X range: [0.0002, 0.9953]
✓ No detection handled correctly (returns None)

✓ TEST 3 PASSED


In [10]:
print("\n" + "="*70)
print("TEST 3b: parse_landmarks_from_results - Wrong Landmark Count")
print("="*70)

try:
    # Create mock with wrong number of landmarks
    landmarks_list = []
    for i in range(100):  # Only 100 instead of 478
        landmark = Mock()
        landmark.x = np.random.rand()
        landmark.y = np.random.rand()
        landmark.z = np.random.rand()
        landmarks_list.append(landmark)

    mock_face_landmarks = Mock()
    mock_face_landmarks.landmark = landmarks_list

    mock_results = Mock()
    mock_results.multi_face_landmarks = [mock_face_landmarks]

    # Parse landmarks - should return None due to count mismatch
    keypoints = parse_landmarks_from_results(mock_results, expected_num=478)

    assert keypoints is None
    print(f"✓ Wrong landmark count handled correctly")
    print(f"  Expected 478, got 100 - returns None")

    print("\n✓ TEST 3b PASSED")
except Exception as e:
    print(f"✗ TEST 3b FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 3b: parse_landmarks_from_results - Wrong Landmark Count
✓ Wrong landmark count handled correctly
  Expected 478, got 100 - returns None

✓ TEST 3b PASSED


In [11]:
print("\n" + "="*70)
print("TEST 4: procrustes_analysis")
print("="*70)

try:
    # Create source and target landmarks
    source = np.random.rand(478, 3).astype(np.float32) * 100
    target = np.random.rand(478, 3).astype(np.float32) * 100

    # Run procrustes
    aligned = procrustes_analysis(source, target)

    assert isinstance(aligned, np.ndarray)
    assert aligned.shape == source.shape
    print(f"✓ procrustes_analysis works")
    print(f"  Source shape: {source.shape}")
    print(f"  Aligned shape: {aligned.shape}")
    print(f"  Alignment successful (shape preserved)")

    print("\n✓ TEST 4 PASSED")
except Exception as e:
    print(f"✗ TEST 4 FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 4: procrustes_analysis
✓ procrustes_analysis works
  Source shape: (478, 3)
  Aligned shape: (478, 3)
  Alignment successful (shape preserved)

✓ TEST 4 PASSED


In [12]:
print("\n" + "="*70)
print("TEST 4c: procrustes_analysis - Zero Norm")
print("="*70)

try:
    # Create zero array (zero norm)
    source = np.zeros((478, 3), dtype=np.float32)
    target = np.random.rand(478, 3).astype(np.float32)

    # Should raise ValueError for zero norm
    try:
        aligned = procrustes_analysis(source, target, num_landmarks=478)
        print(f"✗ Should have raised ValueError for zero norm")
        assert False
    except ValueError as ve:
        if "zero norm" in str(ve).lower():
            print(f"✓ Zero norm raises ValueError correctly")
            print(f"  Error message: {ve}")
        else:
            raise

    print("\n✓ TEST 4c PASSED")
except Exception as e:
    print(f"✗ TEST 4c FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 4c: procrustes_analysis - Zero Norm
✓ Zero norm raises ValueError correctly
  Error message: Zero norm encountered in Procrustes.

✓ TEST 4c PASSED


In [13]:
print("\n" + "="*70)
print("TEST 4b: procrustes_analysis - Shape Mismatch")
print("="*70)

try:
    # Create mismatched shapes
    source = np.random.rand(100, 3).astype(np.float32)
    target = np.random.rand(478, 3).astype(np.float32)

    # Should raise ValueError
    try:
        aligned = procrustes_analysis(source, target, num_landmarks=478)
        print(f"✗ Should have raised ValueError for shape mismatch")
        assert False
    except ValueError as ve:
        if "shape mismatch" in str(ve).lower() or "mismatch" in str(ve).lower():
            print(f"✓ Shape mismatch raises ValueError correctly")
            print(f"  Error message: {ve}")
        else:
            raise

    print("\n✓ TEST 4b PASSED")
except Exception as e:
    print(f"✗ TEST 4b FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 4b: procrustes_analysis - Shape Mismatch
✓ Shape mismatch raises ValueError correctly
  Error message: Procrustes input shapes mismatch.

✓ TEST 4b PASSED


In [14]:
print("\n" + "="*70)
print("TEST 5: center_keypoints")
print("="*70)

try:
    # Create sample keypoints
    keypoints = np.random.rand(478, 3).astype(np.float32) * 200 + 100

    # center_keypoints returns a tuple: (centered, ref_coords)
    centered, ref_coords = center_keypoints(keypoints, reference_index=2)

    assert isinstance(centered, np.ndarray)
    assert isinstance(ref_coords, np.ndarray)
    assert centered.shape == keypoints.shape
    assert ref_coords.shape == (3,)

    # Verify the reference landmark (index 2) is at origin
    assert np.allclose(centered[2], [0, 0, 0], atol=1e-5)

    print(f"✓ center_keypoints works")
    print(f"  Input shape: {keypoints.shape}")
    print(f"  Centered shape: {centered.shape}")
    print(f"  Reference coords shape: {ref_coords.shape}")
    print(f"  Reference landmark (index 2) at origin: {np.allclose(centered[2], [0, 0, 0])}")

    print("\n✓ TEST 5 PASSED")
except Exception as e:
    print(f"✗ TEST 5 FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 5: center_keypoints
✓ center_keypoints works
  Input shape: (478, 3)
  Centered shape: (478, 3)
  Reference coords shape: (3,)
  Reference landmark (index 2) at origin: True

✓ TEST 5 PASSED


In [15]:
print("\n" + "="*70)
print("TEST 5b: center_keypoints - Out of Bounds Index")
print("="*70)

try:
    # Create sample keypoints
    keypoints = np.random.rand(478, 3).astype(np.float32) * 200 + 100

    # Use invalid reference index
    try:
        centered, ref_coords = center_keypoints(keypoints, reference_index=10000)
        print(f"✗ Should have raised IndexError for out of bounds index")
        assert False
    except IndexError:
        print(f"✓ Out of bounds index raises IndexError correctly")
        print(f"  Reference index 10000 is invalid for 478 landmarks")

    print("\n✓ TEST 5b PASSED")
except Exception as e:
    print(f"✗ TEST 5b FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 5b: center_keypoints - Out of Bounds Index
✓ Out of bounds index raises IndexError correctly
  Reference index 10000 is invalid for 478 landmarks

✓ TEST 5b PASSED


In [16]:
print("\n" + "="*70)
print("TEST 6: keypoints_to_feature_vector")
print("="*70)

try:
    # Create sample keypoints (478 x 3)
    keypoints = np.random.rand(478, 3).astype(np.float32) * 200

    # Convert to feature vector
    feature_vector = keypoints_to_feature_vector(keypoints)

    assert isinstance(feature_vector, np.ndarray)
    assert feature_vector.shape[0] == 478 * 3  # Flattened
    assert feature_vector.dtype in [np.float32, np.float64]

    print(f"✓ keypoints_to_feature_vector works")
    print(f"  Input shape: {keypoints.shape}")
    print(f"  Output shape: {feature_vector.shape}")
    print(f"  Flattening successful")

    print("\n✓ TEST 6 PASSED")
except Exception as e:
    print(f"✗ TEST 6 FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 6: keypoints_to_feature_vector
✓ keypoints_to_feature_vector works
  Input shape: (478, 3)
  Output shape: (1434,)
  Flattening successful

✓ TEST 6 PASSED


In [17]:
print("\n" + "="*70)
print("TEST 7: Pipeline Chain Integration")
print("="*70)

try:
    # Create synthetic keypoints
    keypoints = np.random.rand(478, 3).astype(np.float32) * 100

    # Step 1: Center them
    centered, ref_coords = center_keypoints(keypoints, reference_index=2)

    # Step 2: Convert to feature vector
    feature = keypoints_to_feature_vector(centered)

    # Verify the chain
    assert feature.shape == (478 * 3,)
    assert feature.dtype == np.float32
    assert np.allclose(centered[2], [0, 0, 0], atol=1e-6)

    print(f"✓ Pipeline chain works correctly")
    print(f"  keypoints → center → feature_vector")
    print(f"  Input: {keypoints.shape}")
    print(f"  Centered: {centered.shape} (ref point at origin: {np.allclose(centered[2], 0)})")
    print(f"  Feature: {feature.shape}")

    print("\n✓ TEST 7 PASSED")
except Exception as e:
    print(f"✗ TEST 7 FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 7: Pipeline Chain Integration
✓ Pipeline chain works correctly
  keypoints → center → feature_vector
  Input: (478, 3)
  Centered: (478, 3) (ref point at origin: True)
  Feature: (1434,)

✓ TEST 7 PASSED


## Integration Tests

In [19]:
print("\n" + "="*70)
print("TEST 6b: keypoints_to_feature_vector - Different Shapes")
print("="*70)

try:
    # Test with different landmark counts
    test_cases = [
        (478, 3),  # 478 landmarks (MediaPipe format)
    ]

    for n_landmarks, n_dims in test_cases:
        keypoints = np.random.rand(n_landmarks, n_dims).astype(np.float32) * 200
        feature_vector = keypoints_to_feature_vector(keypoints)

        assert isinstance(feature_vector, np.ndarray)
        assert feature_vector.shape[0] == n_landmarks * n_dims
        assert feature_vector.dtype in [np.float32, np.float64]
        print(f"✓ Works for {n_landmarks} landmarks: {keypoints.shape} → {feature_vector.shape}")

    print("\n✓ TEST 6b PASSED")
except Exception as e:
    print(f"✗ TEST 6b FAILED: {e}")
    import traceback
    traceback.print_exc()


TEST 6b: keypoints_to_feature_vector - Different Shapes
✓ Works for 478 landmarks: (478, 3) → (1434,)

✓ TEST 6b PASSED


## Summary

In [20]:
print("\n" + "="*70)
print("MEDIAPIPE PROCESSING PIPELINE TESTS SUMMARY")
print("="*70)
print("✓ TEST 1: load_reference_keypoints - Success")
print("✓ TEST 1b: load_reference_keypoints - Missing File")
print("✓ TEST 1c: load_reference_keypoints - Shape Mismatch")
print("✓ TEST 2: create_face_mesh")
print("✓ TEST 3: parse_landmarks_from_results - Success")
print("✓ TEST 3b: parse_landmarks_from_results - Wrong Count")
print("✓ TEST 4: procrustes_analysis - Success")
print("✓ TEST 4b: procrustes_analysis - Shape Mismatch")
print("✓ TEST 4c: procrustes_analysis - Zero Norm")
print("✓ TEST 5: center_keypoints - Success")
print("✓ TEST 5b: center_keypoints - Out of Bounds")
print("✓ TEST 6: keypoints_to_feature_vector - Success")
print("✓ TEST 6b: keypoints_to_feature_vector - Different Shapes")
print("✓ TEST 7: Pipeline Chain Integration")
print("\n🎉 All MediaPipe pipeline tests completed successfully!")
print("   Total: 14 tests covering success cases and edge cases")
print("="*70)


MEDIAPIPE PROCESSING PIPELINE TESTS SUMMARY
✓ TEST 1: load_reference_keypoints - Success
✓ TEST 1b: load_reference_keypoints - Missing File
✓ TEST 1c: load_reference_keypoints - Shape Mismatch
✓ TEST 2: create_face_mesh
✓ TEST 3: parse_landmarks_from_results - Success
✓ TEST 3b: parse_landmarks_from_results - Wrong Count
✓ TEST 4: procrustes_analysis - Success
✓ TEST 4b: procrustes_analysis - Shape Mismatch
✓ TEST 4c: procrustes_analysis - Zero Norm
✓ TEST 5: center_keypoints - Success
✓ TEST 5b: center_keypoints - Out of Bounds
✓ TEST 6: keypoints_to_feature_vector - Success
✓ TEST 6b: keypoints_to_feature_vector - Different Shapes
✓ TEST 7: Pipeline Chain Integration

🎉 All MediaPipe pipeline tests completed successfully!
   Total: 14 tests covering success cases and edge cases
